In [2]:
import pandas as pd
import numpy as np

In [6]:
filename = 'data.csv'

data = pd.read_csv('data/' + filename)

data['count'] = 1 # counting instances of deaths, 1 death for row

print('date in csv', data['Date'].head(2))
print('type:', type(data['Date']))

# convert date to yyyy-mm-dd
data["Date"] = pd.to_datetime(data['Date'], yearfirst=True, errors='coerce')

data['Date'].head(2)

date in csv 0          NaN
1    2 01 2020
Name: Date, dtype: object
type: <class 'pandas.core.series.Series'>


0          NaT
1   2020-02-01
Name: Date, dtype: datetime64[ns]

### Deaths per Day

In [9]:
# get all instances who died
died = data[data['Status'] == 'Died']

# get count of deaths per day
deaths_per_day = died.groupby(['Date'])['count'].aggregate(sum)
# or
# died.groupby(['Date Died']).agg({'count': ['sum']})

# reset index
deaths_per_day = deaths_per_day.reset_index() # reset columns

### Moving Average

In [10]:
deaths_per_day['7dma'] = deaths_per_day.iloc[:,1].rolling(window=7).mean()
deaths_per_day = deaths_per_day.apply(lambda x: x.fillna('NDA'))


deaths_per_day = deaths_per_day.sort_index(ascending=True) # sort
deaths_per_day

,Date,count,7dma
0,2020-02-01,1,NDA
1,2020-03-11,2,NDA
2,2020-03-12,3,NDA
3,2020-03-13,4,NDA
4,2020-03-14,3,NDA
5,2020-03-15,5,NDA
6,2020-03-16,6,3.42857
7,2020-03-17,11,4.85714
8,2020-03-18,12,6.28571
9,2020-03-19,13,7.71429


### Deats per day Median

In [12]:
died.groupby(['Date'])['count'].aggregate(sum).median()

#max
#min
#unique


9.0

In [13]:
deaths_per_day['count'].median()

# or
# deaths_per_day.quantile(0.5) # same as median

9.0

In [15]:
# Verify Median

un_ded = data.groupby(['Date'])['count'].aggregate(sum)
print(np.median(un_ded))

un_ded = pd.DataFrame(un_ded)
print(un_ded.median())

9.0
count    9.0
dtype: float64


### 75 Percentile

In [16]:
deaths_per_day['count'].quantile(0.75)

15.0

### Prepare for JSON

In [17]:
# formating for JSON
# deaths_per_day = deaths_per_day.reset_index() # reset columns
# deaths_per_day = deaths_per_day.sort_index(ascending=True) # sort

# convert to (date, count) instances
# deaths_per_day = deaths_per_day.to_records().tolist()

# deaths_per_day.to_records(convert_datetime64=False)

deaths_per_day.to_records()[0:2]

rec.array([(0, '2020-02-01T00:00:00.000000000', 1, 'NDA'),
           (1, '2020-03-11T00:00:00.000000000', 2, 'NDA')],
          dtype=[('index', '<i8'), ('Date', '<M8[ns]'), ('count', '<i8'), ('7dma', 'O')])

### Generate Date

In [26]:
sdate = deaths_per_day['Date'][:1] #or .head(1)
edate = deaths_per_day['Date'].tail(1)

sdate = sdate.to_string()[4:]
edate = edate.to_string()[4:]

dates = pd.date_range(sdate, edate, freq='d')


In [27]:
result = pd.DataFrame(dates, columns=['Date'])
result.head(2)

,Date
0,2020-02-01
1,2020-02-02


In [28]:
# join dateframes
res = result.merge(deaths_per_day, on="Date", how = 'outer')

# remonve NaN
res = res.apply(lambda x: x.fillna(0))

res.head()

,Date,count,7dma
0,2020-02-01,1.0,NDA
1,2020-02-02,0.0,0
2,2020-02-03,0.0,0
3,2020-02-04,0.0,0
4,2020-02-05,0.0,0
